In [1]:
from dolfinx.io import VTXWriter
from pathlib import Path
from mpi4py import MPI
from petsc4py.PETSc import ScalarType  # type: ignore
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.fem.petsc import LinearProblem
from ufl import ds, dx, grad, inner

In [2]:
msh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, 0.0), (2.0, 1.0)), n=(32, 16),
                            cell_type=mesh.CellType.triangle)

In [3]:
V = fem.functionspace(msh, ("Lagrange", 1)) # quadratic polynomials

In [4]:
facets = mesh.locate_entities_boundary(msh, dim=(msh.topology.dim - 1),
                                       marker=lambda x: np.logical_or(np.isclose(x[0], 0.0),
                                                                      np.isclose(x[0], 2.0)))

In [5]:
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)  # DOF in the model

In [6]:
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

In [7]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)
f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
g = ufl.sin(5 * x[0])
a = inner(grad(u), grad(v)) * dx
L = inner(f, v) * dx + inner(g, v) * ds

In [8]:
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [9]:
try:
    import pyvista
    cells, types, x = plot.vtk_mesh(V0)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    values = np.zeros((x.shape[0], 3), dtype=np.float64)
    values[:, :msh.topology.dim] = u0.x.array.reshape(x.shape[0], msh.topology.dim).real
    grid.point_data["u"] = values

    pl = pyvista.Plotter(shape=(2, 2))

    pl.subplot(0, 0)
    pl.add_text("magnitude", font_size=12, color="black", position="upper_edge")
    pl.add_mesh(grid.copy(), show_edges=True)

    pl.subplot(0, 1)
    glyphs = grid.glyph(orient="u", factor=0.08)
    pl.add_text("vector glyphs", font_size=12, color="black", position="upper_edge")
    pl.add_mesh(glyphs, show_scalar_bar=False)
    pl.add_mesh(grid.copy(), style="wireframe", line_width=2, color="black")

    pl.subplot(1, 0)
    pl.add_text("x-component", font_size=12, color="black", position="upper_edge")
    pl.add_mesh(grid.copy(), component=0, show_edges=True)

    pl.subplot(1, 1)
    pl.add_text("y-component", font_size=12, color="black", position="upper_edge")
    pl.add_mesh(grid.copy(), component=1, show_edges=True)

    pl.view_xy()
    pl.link_views()

    # If pyvista environment variable is set to off-screen (static)
    # plotting save png
    if pyvista.OFF_SCREEN:
        pyvista.start_xvfb(wait=0.1)
        pl.screenshot("uh.png")
    else:
        pl.show()
except ModuleNotFoundError:
    print("pyvista is required to visualise the solution")

NameError: name 'V0' is not defined